In [1]:
import duckdb
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Companies").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/16 05:33:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Dim Companies

In [35]:
# Connect to the DuckDB database
duck_conn = duckdb.connect('/home/anhcu/Final_ETL_App/etl-app/datawarehouse.duckdb')
query = 'SELECT * FROM dim_companies'

# Fetch data from DuckDB and convert to Pandas DataFrame
pandas_df = duck_conn.execute(query).fetchdf()

duck_conn.close()

In [36]:
# Convert Pandas DataFrame to PySpark DataFrame
spark_df_companies = spark.createDataFrame(pandas_df)

# Show the Spark DataFrame
spark_df_companies.show()

24/06/15 15:56:10 WARN TaskSetManager: Stage 29 contains a task of very large size (12663 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------------------+--------------------+--------------+-------------------+--------------------+----------------+--------------------+---------------------+-------------------+---------------------+-----------------------+--------------------+--------------------+
|company_id|        company_name|  company_time_stamp|company_ticket|company_is_delisted|    company_category|company_currency|    company_location|company_exchange_name|company_region_name|company_industry_name|company_industry_sector|company_sic_industry|  company_sic_sector|
+----------+--------------------+--------------------+--------------+-------------------+--------------------+----------------+--------------------+---------------------+-------------------+---------------------+-----------------------+--------------------+--------------------+
|         1|ADMIRALTY BANCORP...|2024-06-12 13:09:...|          AAAB|               true|Domestic Common S...|             USD|      Florida; U.S.A|               

In [37]:
from pyspark.sql.functions import col, count

In [38]:
# Thống kê số lượng công ty ở từng sàn giao dịch
print("Thống kê số lượng công ty ở từng sàn giao dịch:")
spark_df_companies.groupBy("company_exchange_name").agg(count("*").alias("count")).show()

Thống kê số lượng công ty ở từng sàn giao dịch:


24/06/15 15:56:14 WARN TaskSetManager: Stage 30 contains a task of very large size (12663 KiB). The maximum recommended task size is 1000 KiB.


+---------------------+-----+
|company_exchange_name|count|
+---------------------+-----+
|                 NYSE|39297|
|               NASDAQ|78894|
+---------------------+-----+



In [39]:
# Thống kê số lượng công ty đã và chưa bị delisted
print("Thống kê số lượng công ty đã và chưa bị delisted:")
spark_df_companies.groupBy("company_is_delisted").agg(count("*").alias("count")).show()

Thống kê số lượng công ty đã và chưa bị delisted:


24/06/15 15:56:16 WARN TaskSetManager: Stage 33 contains a task of very large size (12663 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+-----+
|company_is_delisted|count|
+-------------------+-----+
|               true|79107|
|              false|39084|
+-------------------+-----+



In [40]:
# Thống kê số lượng công ty theo khu vực
print("Thống kê số lượng công ty theo khu vực:")
spark_df_companies.groupBy("company_location").agg(count("*").alias("count")).orderBy(col("count").desc()).show()

Thống kê số lượng công ty theo khu vực:


24/06/15 15:56:19 WARN TaskSetManager: Stage 36 contains a task of very large size (12663 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|    company_location|count|
+--------------------+-----+
|   California; U.S.A|18708|
|     New York; U.S.A|13425|
|        Texas; U.S.A| 8883|
|Massachusetts; U.S.A| 5664|
|      Florida; U.S.A| 5130|
| Pennsylvania; U.S.A| 3912|
|     Illinois; U.S.A| 3750|
|   New Jersey; U.S.A| 3690|
|               China| 3150|
|      Georgia; U.S.A| 2625|
|     Virginia; U.S.A| 2592|
|     Colorado; U.S.A| 2580|
|         Ohio; U.S.A| 2340|
|  Connecticut; U.S.A| 2208|
|    Minnesota; U.S.A| 1992|
|North Carolina; U...| 1902|
|     Maryland; U.S.A| 1884|
|      United Kingdom| 1638|
|   Washington; U.S.A| 1620|
|     Ontario; Canada| 1422|
+--------------------+-----+
only showing top 20 rows



In [41]:
# Thống kê số lượng công ty theo ngành công nghiệp
print("Thống kê số lượng công ty theo ngành công nghiệp:")
spark_df_companies.groupBy("company_industry_name").agg(count("*").alias("count")).orderBy(col("count").desc()).show()

Thống kê số lượng công ty theo ngành công nghiệp:


24/06/15 15:56:22 WARN TaskSetManager: Stage 39 contains a task of very large size (12663 KiB). The maximum recommended task size is 1000 KiB.


+---------------------+-----+
|company_industry_name|count|
+---------------------+-----+
|      Shell Companies|10797|
| Software - Applic...| 9774|
|        Biotechnology| 9282|
|     Banks - Regional| 9270|
|      Medical Devices| 3000|
|     Telecom Services| 2973|
| Communication Equ...| 2394|
|        Oil & Gas E&P| 2196|
|      REIT - Mortgage| 1932|
|       Semiconductors| 1698|
|     Asset Management| 1650|
| Software - Infras...| 1596|
|     Specialty Retail| 1404|
| Medical Care Faci...| 1368|
| Specialty Industr...| 1302|
|    Computer Hardware| 1260|
|  Oil & Gas Midstream| 1218|
|        Entertainment| 1200|
| Internet Content ...| 1194|
|       Packaged Foods| 1194|
+---------------------+-----+
only showing top 20 rows



In [42]:
# Thống kê số lượng công ty theo ngành công nghiệp sector
sector_stats = spark_df_companies.groupBy("company_industry_sector").agg(count("*").alias("count")).orderBy(col("count").desc())
sector_stats.show()

24/06/15 15:56:24 WARN TaskSetManager: Stage 42 contains a task of very large size (12663 KiB). The maximum recommended task size is 1000 KiB.


+-----------------------+-----+
|company_industry_sector|count|
+-----------------------+-----+
|            Industrials|22113|
|             Technology|20346|
|             Healthcare|17934|
|     Financial Services|17322|
|      Consumer Cyclical|11412|
|   Communication Ser...| 6699|
|            Real Estate| 5910|
|                 Energy| 5067|
|        Basic Materials| 4902|
|     Consumer Defensive| 4284|
|              Utilities| 2202|
+-----------------------+-----+



In [43]:
# Thống kê số lượng công ty theo loại hình công ty
category_stats = spark_df_companies.groupBy("company_category").agg(count("*").alias("count")).orderBy(col("count").desc())
category_stats.show()

24/06/15 15:56:27 WARN TaskSetManager: Stage 45 contains a task of very large size (12663 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|    company_category|count|
+--------------------+-----+
|Domestic Common S...|73089|
|Domestic Common S...|10854|
|    ADR Common Stock| 8892|
|Domestic Common S...| 7776|
|Domestic Common S...| 6168|
|Domestic Preferre...| 6150|
|Canadian Common S...| 1482|
|ADR Common Stock ...| 1326|
|ADR Common Stock ...| 1074|
|ADR Common Stock ...|  732|
| ADR Preferred Stock|  534|
|Canadian Common S...|   60|
|Canadian Common S...|   24|
|Canadian Preferre...|   18|
|Canadian Common S...|   12|
+--------------------+-----+



## Fact Candles

In [3]:
import pandas as pd
from datetime import datetime
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, DoubleType, BooleanType, TimestampType
from pyspark.sql.functions import count
from pyspark.sql.functions import avg
from pyspark.sql.functions import col, expr

In [8]:
# Connect to the DuckDB database
duck_conn = duckdb.connect('/home/anhcu/Final_ETL_App/etl-app/datawarehouse.duckdb')
query = 'SELECT * FROM fact_candles'

# Fetch data from DuckDB and convert to Pandas DataFrame
pandas_df = duck_conn.execute(query).fetchdf()

duck_conn.close()

In [9]:
pandas_df

,candle_id,candle_company_id,candle_volume,candle_volume_weighted,candle_open,candle_close,candle_high,candle_low,candle_time_stamp,candle_num_of_trades,candle_is_otc,candles_time_id
0,48735,22127,29174,48.1472,48.270,48.300,48.300,47.7500,1718136000000,1819,NaN,1
1,48736,26716,1288577,37.9410,37.880,38.090,38.170,37.5000,1718136000000,18317,NaN,1
2,48737,36670,706413,44.6400,45.550,44.730,45.555,44.2700,1718136000000,11110,NaN,1
3,48738,30065,84449,0.7683,0.800,0.751,0.810,0.7401,1718136000000,290,NaN,1
4,48739,33016,8024,209.2152,210.270,208.950,210.270,208.9500,1718136000000,1330,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
24362,73097,104194,102,10.7613,10.760,10.760,10.760,10.7600,1718395200000,3,NaN,4
24363,73098,102886,561,16.4572,16.490,16.490,16.490,16.4900,1718395200000,12,NaN,4
24364,73099,102735,2100,29.9410,30.000,30.000,30.000,30.0000,1718395200000,2,True,4
24365,73100,110274,100,10.5700,10.570,10.570,10.570,10.5700,1718395200000,1,NaN,4


In [25]:
# Loại bỏ các cột không cần thiết
pandas_df.drop(columns=['candles_time_id'], inplace=True)

# Chuyển đổi kiểu dữ liệu
pandas_df['candle_is_otc'] = pandas_df['candle_is_otc'].fillna(False).astype(bool)
pandas_df['candle_time_stamp'] = pd.to_datetime(pandas_df['candle_time_stamp'], unit='ms')

pandas_df

/tmp/ipykernel_49482/1880719106.py:6: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  pandas_df['candle_time_stamp'] = pd.to_datetime(pandas_df['candle_time_stamp'], unit='ms')


,candle_id,candle_company_id,candle_volume,candle_volume_weighted,candle_open,candle_close,candle_high,candle_low,candle_time_stamp,candle_num_of_trades,candle_is_otc,candle_percent_change_open_close,candle_percent_change_high_low
0,24368,22127,29174,48.1472,48.270,48.300,48.300,47.7500,2024-06-11 19:59:07.264,1819,False,0.062150,1.151832
1,24369,26716,1288577,37.9410,37.880,38.090,38.170,37.5000,2024-06-11 19:59:07.264,18317,False,0.554382,1.786667
2,24370,36670,706413,44.6400,45.550,44.730,45.555,44.2700,2024-06-11 19:59:07.264,11110,False,-1.800220,2.902643
3,24371,30065,84449,0.7683,0.800,0.751,0.810,0.7401,2024-06-11 19:59:07.264,290,False,-6.125000,9.444670
4,24372,33016,8024,209.2152,210.270,208.950,210.270,208.9500,2024-06-11 19:59:07.264,1330,False,-0.627764,0.631730
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24362,48730,104194,102,10.7613,10.760,10.760,10.760,10.7600,2024-06-14 20:00:07.680,3,False,0.000000,0.000000
24363,48731,102886,561,16.4572,16.490,16.490,16.490,16.4900,2024-06-14 20:00:07.680,12,False,0.000000,0.000000
24364,48732,102735,2100,29.9410,30.000,30.000,30.000,30.0000,2024-06-14 20:00:07.680,2,True,0.000000,0.000000
24365,48733,110274,100,10.5700,10.570,10.570,10.570,10.5700,2024-06-14 20:00:07.680,1,False,0.000000,0.000000


In [26]:
# Xác định schema
schema = StructType([
    StructField("candle_id", IntegerType(), True),
    StructField("candle_company_id", IntegerType(), True),
    StructField("candle_volume", LongType(), True),
    StructField("candle_volume_weighted", DoubleType(), True),
    StructField("candle_open", DoubleType(), True),
    StructField("candle_close", DoubleType(), True),
    StructField("candle_high", DoubleType(), True),
    StructField("candle_low", DoubleType(), True),
    StructField("candle_time_stamp", TimestampType(), True),
    StructField("candle_num_of_trades", IntegerType(), True),
    StructField("candle_is_otc", BooleanType(), True),
    StructField("candle_percent_change_open_close", DoubleType(), True),
    StructField("candle_percent_change_high_low", DoubleType(), True)
])

# Chuyển đổi Pandas DataFrame thành PySpark DataFrame với schema
spark_df_candles = spark.createDataFrame(pandas_df, schema=schema)

# Hiển thị một vài hàng của DataFrame
spark_df_candles.show()

+---------+-----------------+-------------+----------------------+-----------+------------+-----------+----------+--------------------+--------------------+-------------+--------------------------------+------------------------------+
|candle_id|candle_company_id|candle_volume|candle_volume_weighted|candle_open|candle_close|candle_high|candle_low|   candle_time_stamp|candle_num_of_trades|candle_is_otc|candle_percent_change_open_close|candle_percent_change_high_low|
+---------+-----------------+-------------+----------------------+-----------+------------+-----------+----------+--------------------+--------------------+-------------+--------------------------------+------------------------------+
|    24368|            22127|        29174|               48.1472|      48.27|        48.3|       48.3|     47.75|2024-06-11 19:59:...|                1819|        false|             0.06215040397761349|            1.1518324607329784|
|    24369|            26716|      1288577|                3

24/06/16 03:27:25 WARN TaskSetManager: Stage 1 contains a task of very large size (1077 KiB). The maximum recommended task size is 1000 KiB.


In [52]:
# Thống kê số lượng giao dịch
trade_stats = spark_df_candles.groupBy("candle_is_otc").agg(count("*").alias("count"))
trade_stats.show()

+-------------+-----+
|candle_is_otc|count|
+-------------+-----+
|         true| 1423|
|        false|22944|
+-------------+-----+



In [54]:
# Thống kê khối lượng giao dịch trung bình theo công ty
volume_avg_stats = spark_df_candles.groupBy("candle_company_id").agg(avg("candle_volume").alias("avg_volume"))
volume_avg_stats.show()

+-----------------+----------+
|candle_company_id|avg_volume|
+-----------------+----------+
|            26087|  342077.0|
|            37111|  423012.0|
|            20497|  544030.0|
|            31236|   24725.0|
|            35820|  497481.0|
|            36131| 1347812.0|
|            29993|   10163.0|
|            29719|   93199.0|
|            29601|  709573.0|
|            38422|   30176.0|
|            35689|  273380.0|
|            19984|  389931.0|
|            20135|   79299.0|
|            38311|  457856.0|
|            35071|  360757.0|
|            35947|  713053.0|
|            36538| 1359825.0|
|            20382|  552118.0|
|            29228|  535699.0|
|            29894| 2045242.0|
+-----------------+----------+
only showing top 20 rows



In [55]:
# Thống kê giá mở cửa trung bình theo công ty
open_avg_stats = spark_df_candles.groupBy("candle_company_id").agg(avg("candle_open").alias("avg_open"))
open_avg_stats.show()

+-----------------+--------+
|candle_company_id|avg_open|
+-----------------+--------+
|            26087|    2.47|
|            37111|  161.78|
|            20497|     1.1|
|            31236|   0.065|
|            35820|   45.08|
|            36131|    44.0|
|            29993|     7.3|
|            29719|   25.11|
|            29601|    7.87|
|            38422|    4.66|
|            35689|    8.02|
|            19984|   17.16|
|            20135|    3.24|
|            38311|   25.11|
|            35071|   58.52|
|            35947|   78.32|
|            36538|   77.52|
|            20382|   31.95|
|            29228|  169.61|
|            29894|   92.79|
+-----------------+--------+
only showing top 20 rows



## Mix

In [56]:
# Kết hợp hai bảng dựa trên company_id và candle_company_id
joined_df = spark_df_candles.join(spark_df_companies, spark_df_candles.candle_company_id == spark_df_companies.company_id)

# Hiển thị một vài hàng của DataFrame kết hợp
joined_df.show()

24/06/15 16:04:39 WARN TaskSetManager: Stage 59 contains a task of very large size (12663 KiB). The maximum recommended task size is 1000 KiB.


+---------+-----------------+-------------+----------------------+-----------+------------+-----------+----------+--------------------+--------------------+-------------+----------+--------------------+--------------------+--------------+-------------------+--------------------+----------------+--------------------+---------------------+-------------------+---------------------+-----------------------+--------------------+--------------------+
|candle_id|candle_company_id|candle_volume|candle_volume_weighted|candle_open|candle_close|candle_high|candle_low|   candle_time_stamp|candle_num_of_trades|candle_is_otc|company_id|        company_name|  company_time_stamp|company_ticket|company_is_delisted|    company_category|company_currency|    company_location|company_exchange_name|company_region_name|company_industry_name|company_industry_sector|company_sic_industry|  company_sic_sector|
+---------+-----------------+-------------+----------------------+-----------+------------+-----------+-

In [57]:
from pyspark.sql.functions import avg

# Thống kê khối lượng giao dịch trung bình theo ngành công nghiệp
volume_avg_industry_stats = joined_df.groupBy("company_industry_name").agg(avg("candle_volume").alias("avg_volume")).orderBy("avg_volume", ascending=False)
volume_avg_industry_stats.show()

24/06/15 16:05:02 WARN TaskSetManager: Stage 65 contains a task of very large size (12663 KiB). The maximum recommended task size is 1000 KiB.


+---------------------+------------------+
|company_industry_name|        avg_volume|
+---------------------+------------------+
|   Auto Manufacturers|  9333003.97044335|
| Consumer Electronics| 7793233.613333333|
|       Semiconductors| 6382139.212624584|
|               Silver|         5820558.5|
|  Banks - Diversified|         5676967.6|
| Drug Manufacturer...|  4567538.97368421|
|  Beverages - Brewers| 4463199.230769231|
| Oil & Gas Integrated| 4434751.106666666|
|  Aerospace & Defense| 4215876.419161676|
|             Airlines|4039972.3863636362|
|      Capital Markets| 3946098.194174757|
|     Specialty Retail| 3552747.336492891|
|    Resorts & Casinos|3384854.0632911394|
|        Entertainment|3353888.1428571427|
|                 Gold| 3314450.192982456|
|              Tobacco| 3297863.076923077|
|      Travel Services| 3284076.289473684|
|           Publishing|3082967.1578947366|
|               Copper|         2860272.2|
|      Internet Retail| 2783201.552631579|
+----------

In [58]:
# Thống kê giá đóng cửa trung bình theo thời gian
close_avg_region_stats = joined_df.groupBy("candle_time_stamp").agg(avg("candle_close").alias("avg_close")).orderBy("avg_close", ascending=False)
close_avg_region_stats.show()

24/06/15 16:06:06 WARN TaskSetManager: Stage 74 contains a task of very large size (12663 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+------------------+
|   candle_time_stamp|         avg_close|
+--------------------+------------------+
|2024-06-12 20:00:...|146.70842157243322|
|2024-06-11 19:59:...| 144.6122471598025|
|2024-06-14 20:00:...|143.70532500131725|
|2024-06-13 20:00:...|143.40294943669167|
+--------------------+------------------+



In [59]:
# Thống kê số lượng công ty theo ngành công nghiệp và sàn giao dịch
company_count_industry_exchange_stats = joined_df.groupBy("company_industry_name", "company_exchange_name").agg(count("company_id").alias("num_companies")).orderBy("num_companies", ascending=False)
company_count_industry_exchange_stats.show()

24/06/15 16:06:34 WARN TaskSetManager: Stage 83 contains a task of very large size (12663 KiB). The maximum recommended task size is 1000 KiB.


+---------------------+---------------------+-------------+
|company_industry_name|company_exchange_name|num_companies|
+---------------------+---------------------+-------------+
|        Biotechnology|               NASDAQ|         2865|
|     Banks - Regional|               NASDAQ|         1303|
| Software - Applic...|               NASDAQ|          842|
|      Shell Companies|               NASDAQ|          835|
|      Medical Devices|               NASDAQ|          558|
| Software - Infras...|               NASDAQ|          514|
|     Banks - Regional|                 NYSE|          331|
| Drug Manufacturer...|               NASDAQ|          326|
| Software - Applic...|                 NYSE|          320|
| Health Informatio...|               NASDAQ|          265|
|       Semiconductors|               NASDAQ|          249|
| Communication Equ...|               NASDAQ|          245|
|     Asset Management|               NASDAQ|          244|
|      Capital Markets|               NA